# Run fine-tuning job

This is the code I used to fine-tune a model using the OpenAI API.

### Imports and credentials

First I have to load the credentials for my OpenAI account. I don't store those in the repo for obvious reasons!

In [1]:
import openai

with open('../credentials.txt', encoding = 'utf-8') as f:
    organization = f.readline().strip()
    api_key = f.readline().strip()
    
client = openai.OpenAI(organization=organization, api_key=api_key)

import pprint

### Upload training and validation files

We created these files in CreateFineTuningData.ipynb.

Now we upload them to OpenAI, and *make a note of the file IDs for later use*.

In [29]:
response = client.files.create(
  file=open("../fine_tuning_data_synopsis_detection_train.jsonl", "rb"),
  purpose="fine-tune"
)

In [30]:
pprint.pprint(response)


FileObject(id='file-DnAhG7uZnSwBo7pgva3Qj5', bytes=3018480, created_at=1762970233, filename='fine_tuning_data_synopsis_detection_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


In [31]:
response2 = client.files.create(
  file=open("../fine_tuning_data_synopsis_detection_val.jsonl", "rb"),
  purpose="fine-tune"
)

In [32]:
pprint.pprint(response2)

FileObject(id='file-N8dS8cypeLbkjXJ3crTg4J', bytes=800540, created_at=1762970242, filename='fine_tuning_data_synopsis_detection_val.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)


### The actual fine-tuning job

This starts the fine-tuning job itself. Again, we make a note of the job ID for future reference.

In [55]:
job = client.fine_tuning.jobs.create(
    training_file='file-DnAhG7uZnSwBo7pgva3Qj5',
    validation_file='file-N8dS8cypeLbkjXJ3crTg4J',
    model="gpt-4.1-mini-2025-04-14"
)
job  

FineTuningJob(id='ftjob-K9tz2rKsUcQ0S4fYdG7xeUOJ', created_at=1762974663, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4.1-mini-2025-04-14', object='fine_tuning.job', organization_id='org-aw09ch9csbZw51lKeR83xQAq', result_files=[], seed=119428093, status='validating_files', trained_tokens=None, training_file='file-DnAhG7uZnSwBo7pgva3Qj5', validation_file='file-N8dS8cypeLbkjXJ3crTg4J', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'))), user_provided_suffix=None, usage_metrics=None, shared_with_openai=False, eval_id=None)

In [ ]:
# If you need to cancel for some reason:

# response = client.fine_tuning.jobs.cancel('ftjob-xLmXQwZZMhA7sO72FJArp55H')
response.status

'cancelled'

In [ ]:
# Checking on the progress of the job:

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id='ftjob-K9tz2rKsUcQ0S4fYdG7xeUOJ', limit=2400)
pprint.pprint(events)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-J6nOeWYndfxWwhAH6cXV7kFk', created_at=1762977014, level='info', message='Evaluating model against our usage policies', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-v4GKxSFDcBlJtRpOfAgFQ3aT', created_at=1762977014, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-Nhz5zLhyjqsPg2hNUQM2UAWT', created_at=1762977014, level='info', message='Checkpoint created at step 1546', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-78rS50XGcRKik7FxGwtlUH5R', created_at=1762977014, level='info', message='Checkpoint created at step 773', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-ib1g4AuMWJhXhnbofcijfeX1', created_at=1762976976, level='info', message='Step 2319/2319: training loss=0.00, validation loss=0.00, full 

In [67]:
# Function to format and print events
def format_events(events):
    for event in events.data:
        print(f"Message: {event.message}")
        if event.data:
            print("Metrics:")
            for key, value in event.data.items():
                print(f"  {key}: {value}")

format_events(events)

Message: Evaluating model against our usage policies
Message: New fine-tuned model created
Message: Checkpoint created at step 1546
Message: Checkpoint created at step 773
Message: Step 2319/2319: training loss=0.00, validation loss=0.00, full validation loss=0.00
Metrics:
  step: 2319
  train_loss: 0.00013566017150878906
  valid_loss: 0.00016736984252929688
  total_steps: 2319
  full_valid_loss: 0.003698784996481503
  train_mean_token_accuracy: 1.0
  valid_mean_token_accuracy: 1.0
  full_valid_mean_token_accuracy: 0.9990588235294118
Message: Step 2318/2319: training loss=0.00
Metrics:
  step: 2318
  train_loss: 0.00015664100646972656
  total_steps: 2319
  train_mean_token_accuracy: 1.0
Message: Step 2317/2319: training loss=0.44
Metrics:
  step: 2317
  train_loss: 0.44483888149261475
  total_steps: 2319
  train_mean_token_accuracy: 0.9333333333333333
Message: Step 2316/2319: training loss=0.00
Metrics:
  step: 2316
  train_loss: 0.00016921758651733398
  total_steps: 2319
  train_mean_

### Getting MODEL ID

After fine-tuning is complete. It can take almost an hour from the last step to model registration.

In [73]:
# Another way to check status

# Note that in the top line here I get the model ID

for j in client.fine_tuning.jobs.list(limit=10).data:
    print(j.id, j.status, j.fine_tuned_model)

ftjob-K9tz2rKsUcQ0S4fYdG7xeUOJ succeeded ft:gpt-4.1-mini-2025-04-14:tedunderwood::CbB93x2c
ftjob-jHTZofcBszm4Y6j145UuBNnv cancelled None
ftjob-xLmXQwZZMhA7sO72FJArp55H cancelled None
ftjob-ovpqNAihOMRRduIKvNy5aNxl succeeded ft:gpt-4.1-mini-2025-04-14:tedunderwood::Cb6Xi4Cm
ftjob-srsB9IEdxVIroirBgKlsSmIa succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold4:AnBj7utC
ftjob-h55W7U720wbURqnTwFTSg547 succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold4:AnCFFewu
ftjob-gV8jBssnqSSktIQ1Pim6OcMA succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold1:AnA6FMqJ
ftjob-LAPh0VES4bqdRtOKSrge2YcA succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold1:An75ZEGH
ftjob-NNHA92xUdDEyJHPWysrvXlyW succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold1:AmaWUOgb
ftjob-Bn8fadk381gdpTXKZyzDOQ03 succeeded ft:gpt-4o-mini-2024-07-18:tedunderwood:fold0:AmVmtrXW


In [69]:
detail = client.fine_tuning.jobs.retrieve('ftjob-K9tz2rKsUcQ0S4fYdG7xeUOJ')
print(detail.status, detail.fine_tuned_model)

running None


### Old legacy code from a year ago below

In [60]:
client.fine_tuning.jobs.retrieve("ftjob-NNHA92xUdDEyJHPWysrvXlyW")

FineTuningJob(id='ftjob-NNHA92xUdDEyJHPWysrvXlyW', created_at=1736142422, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:tedunderwood:fold1:AmaWUOgb', finished_at=1736143500, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-aw09ch9csbZw51lKeR83xQAq', result_files=['file-LbF6eitev3fk8EKFUsQueJ'], seed=465875889, status='succeeded', trained_tokens=256119, training_file='file-G5hQ6ECGsx981bMKH5Txny', validation_file='file-DMnvsN3abZhu3huz5fUYGD', estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix='fold1')

In [72]:
result_file_id = "file-LbF6eitev3fk8EKFUsQueJ"
file_info = client.files.retrieve(result_file_id)
print(file_info)

FileObject(id='file-LbF6eitev3fk8EKFUsQueJ', bytes=21216, created_at=1736143504, filename='step_metrics.csv', object='file', purpose='fine-tune-results', status='processed', status_details=None)


In [2]:
file_content = client.files.download(file_id)

# Save the content to a local file
with open("result_file_fold1.jsonl", "wb") as file:
    file.write(file_content)

AttributeError: 'Files' object has no attribute 'download'